In [7]:
import cv2
import numpy as np

def multi_scale_template_matching(image, template, mask=None, method=cv2.TM_CCOEFF_NORMED, 
                                scale_range=(0.5, 1.5), steps=10):
    best_scale, best_score, best_loc = None, -1, None
    template_h, template_w = template.shape[:2]
    
    for scale in np.linspace(scale_range[0], scale_range[1], steps):
        # Resize the image
        resized = cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
        
        # Skip if template is larger than resized image
        if resized.shape[0] < template_h or resized.shape[1] < template_w:
            continue
            
        # Match template
        if mask is not None:
            result = cv2.matchTemplate(resized, template, method, mask=mask)
        else:
            result = cv2.matchTemplate(resized, template, method)
        
        # For SQDIFF methods, we look for minimum, others look for maximum
        if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
            _, max_val, _, min_loc = cv2.minMaxLoc(result)
            current_val = max_val
            current_loc = min_loc
        else:
            _, max_val, _, max_loc = cv2.minMaxLoc(result)
            current_val = max_val
            current_loc = max_loc
        
        # Update best match
        if current_val > best_score:
            best_scale = scale
            best_score = current_val
            best_loc = current_loc
            best_size = (template_w, template_h)
    
    if best_scale:
        # Scale the bounding box back to original image coordinates
        orig_loc = (int(best_loc[0] / best_scale), int(best_loc[1] / best_scale))
        orig_size = (int(best_size[0] / best_scale), int(best_size[1] / best_scale))
        return orig_loc, orig_size, best_score
    return None, None, 0

# Load images
template = cv2.imread('Finding/3-tiger.jpg', cv2.IMREAD_GRAYSCALE)
large_img = cv2.imread('Finding/3-clean.jpg', cv2.IMREAD_GRAYSCALE)

# Check if images loaded successfully
if template is None or large_img is None:
    print("Error: Could not load images.")
    exit()

# Create mask for white background
_, mask = cv2.threshold(template, 250, 255, cv2.THRESH_BINARY_INV)

# All available template matching methods
methods = {
    'TM_CCOEFF': cv2.TM_CCOEFF,
    'TM_CCOEFF_NORMED': cv2.TM_CCOEFF_NORMED,
    'TM_CCORR': cv2.TM_CCORR,
    'TM_CCORR_NORMED': cv2.TM_CCORR_NORMED,
    'TM_SQDIFF': cv2.TM_SQDIFF,
    'TM_SQDIFF_NORMED': cv2.TM_SQDIFF_NORMED
}

# Display settings
scale_percent = 30 # Smaller display size (40% of original)
wait_time = 2000    # 2 seconds between methods (press any key to continue faster)

for name, method in methods.items():
    # Apply multi-scale template matching
    location, size, confidence = multi_scale_template_matching(
        large_img, template, mask, method,
        scale_range=(0.7, 1.4), steps=20
    )
    
    if location is None:
        print(f"{name}: No match found")
        continue
    
    # Draw rectangle on copy of original image
    display_img = cv2.cvtColor(large_img, cv2.COLOR_GRAY2BGR)
    top_left = (int(location[0]), int(location[1]))
    bottom_right = (int(location[0] + size[0]), int(location[1] + size[1]))
    cv2.rectangle(display_img, top_left, bottom_right, (0, 255, 0), 2)
    
    # Resize for smaller display
    width = int(display_img.shape[1] * scale_percent / 100)
    height = int(display_img.shape[0] * scale_percent / 100)
    resized_img = cv2.resize(display_img, (width, height), interpolation=cv2.INTER_AREA)
    
    # Add method info
    text1 = f"Method: {name}"
    text2 = f"Confidence: {confidence:.4f}"
    text3 = f"Scale: {size[0]/template.shape[1]:.2f}x"
    cv2.putText(resized_img, text1, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    cv2.putText(resized_img, text2, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    cv2.putText(resized_img, text3, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    
    # Display result
    cv2.imshow('Multi-Scale Template Matching', resized_img)
    
    # Wait for key press or timeout
    cv2.waitKey(0)
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [1]:
import cv2
import numpy as np
from collections import Counter

# Load the image (object with background)
img = cv2.imread("Finding/1-cone.jpg")  # BGR format
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB for better interpretation

# Flatten the image to a list of pixels
pixels = img_rgb.reshape(-1, 3)

# Find the most common color (background)
bg_color = Counter(map(tuple, pixels)).most_common(1)[0][0]
print(f"Dominant (Background) Color (RGB): {bg_color}")
# Get corner pixels (top-left, top-right, bottom-left, bottom-right)
corners = [
    img_rgb[0, 0], img_rgb[0, -1],  # Top-left, Top-right
    img_rgb[-1, 0], img_rgb[-1, -1]   # Bottom-left, Bottom-right
]

# Find the most common corner color (background)
bg_color = Counter(map(tuple, corners)).most_common(1)[0][0]
print(f"Background Color (RGB): {bg_color}")

Dominant (Background) Color (RGB): (np.uint8(254), np.uint8(254), np.uint8(254))
Background Color (RGB): (np.uint8(255), np.uint8(255), np.uint8(255))


In [6]:
import cv2
import numpy as np

# Load template (grayscale)
template = cv2.imread('Finding/1-cone.jpg', cv2.IMREAD_GRAYSCALE)

# Create mask (white background = ignored)
_, mask = cv2.threshold(template, 240, 255, cv2.THRESH_BINARY_INV)

# Stack images horizontally for comparison
result_visual = np.hstack((template, mask))

# Display
cv2.imshow('Template (Left) vs. Mask (Right)', result_visual)
cv2.waitKey(0)
cv2.destroyAllWindows()